### Custom GP regressor

In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import WhiteKernel
from numpy import array

class GaussianProcessRegressor_(BaseEstimator, ClassifierMixin):
    #standard init function
    def __init__(self, nu = 1.5):
        self.nu = nu
        return None
    #creates an explicit pipeline so that special features of the gp regressor can be accessed
    def fit(self, X, y, dy = None):
        #transformers
        self.standard_scaler_ = StandardScaler()
        self.standard_scaler_y_ = StandardScaler()
        #predictor
        dimensions = len(X[0])
        kernel = (Matern(length_scale = [1.0 for axis in range(dimensions)], nu = self.nu) 
                  + WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-05, 100000.0)))
        if dy is None:
            self.gaussian_process_ = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer = 10)
        else:
            self.gaussian_process_ = GaussianProcessRegressor(kernel = kernel, alpha = dy, n_restarts_optimizer = 10)
        #transform and fit
        X = self.standard_scaler_.fit_transform(X)
        y = array(y)
        y.shape = (len(y), 1)
        y = self.standard_scaler_y_.fit_transform(y)
        y.shape = (1, len(y))
        y = y[0]
        self.gaussian_process_.fit(X, y)
        return self
    #predict the thing and inverse transform the predicted scaled output  
    def predict(self, X, y = None, return_std = False):
        X = self.standard_scaler_.transform(X)
        y_predict_scaled = self.gaussian_process_.predict(X)
        if not return_std:
            y_predict_scaled = self.gaussian_process_.predict(X)
            y_predict = self.standard_scaler_y_.inverse_transform(y_predict_scaled)
            return y_predict
        else:
            y_predict_scaled, y_std_scaled = self.gaussian_process_.predict(X, return_std = True)
            y_predict = self.standard_scaler_y_.inverse_transform(y_predict_scaled)
            y_upper = self.standard_scaler_y_.inverse_transform(y_predict_scaled + y_std_scaled)
            return y_predict, (y_upper - y_predict)

### Optimization wrapper for GP regressor

In [189]:
#this is the protocol for storing the bayesian optimizer object
#from gaussian_process_regressor_ import GaussianProcessRegressor_
import dill
#from scipy.optimize import anneal
from scipy.optimize import minimize
from numpy import array
from random import random
from numpy import exp, pi, sqrt
from scipy.special import erf

class BayesianOptimizer:
    #if a previous bayesian optimzer was made it can be loaded
    #else we do not do anything
    def __init__(self, filename = None):
        return None     
    #establish the regressor        
    def InitalizeRegressor(self, nu = 1.5):
        self.gaussian_process_regressor_ = GaussianProcessRegressor_(nu = nu)
        return None   
    #set the various simulated annealing parameters    
    def InitializeOptimizer(self, rand_starts=1000, tol=1e-10):
        self.rand_starts_ = rand_starts
        self.tol_ = tol
        None
    #sets the maximum fitness value based on either standard or knowledge gradient
    def SetMaxFitness(self):
        self.y_max_real_and_virtual_ = max(self.y_ + self.y_virtual_)
        return None
    #establish a library of initial samples to seed the model with
    def InitialSamples(self, features, fitness):
        self.X_ = features
        self.y_ = fitness
        self.X_virtual_ = []
        self.y_virtual_ = []
        self.SetMaxFitness()
        return None
    #adds samples - typically this is a result of doing new experiments
    def AddSamples(self, features, fitness):
        self.X_ = self.X_ + features
        self.y_ = self.y_ + fitness
        self.SetMaxFitness()
        return None
    #this controls the range of valid possible values for each axis
    def SetRanges(self, ranges):
        self.ranges_ = ranges
        return None
    #adds some virtual samples - this is relevant to multipoint optimization
    def AddVirtualSamples(self, features, fitness):
        self.X_virtual_ = self.X_virtual_ + features
        self.y_virtual_ = self.y_virtual_ + fitness
        self.SetMaxFitness()
        return None
    #the fitness landscape to optimize and find the best fitness
    def ExpectedImprovement(self, x):
        f_mean, f_stdev  = self.gaussian_process_regressor_.predict(array([x]), return_std = True)
        f_mean = f_mean[0]
        f_stdev = f_stdev[0]
        f_max = self.y_max_real_and_virtual_
        f1 = exp(-((f_max - f_mean)**2.0)/(2.0*f_stdev**2.0))/(2.0*pi)
        f2 = (f_mean - f_max)/(f_stdev*2.0*sqrt(2.0*pi))
        f3 = (f_max - f_mean)*erf((f_max - f_mean)/(sqrt(2.0)*f_stdev))/(2.0*sqrt(2.0*pi)*f_stdev)
        f = f1 + f2 + f3
        #print f
        return f    
    #just the negative of the expected improvement
    def NegativeExpectedImprovement(self, x):
        return -1.0*self.ExpectedImprovement(x)
    #wrapper for the scipy l-bfgs-b optimizer
    #perform some number of random initializations and take the best result    
    def Optimize(self):
        rand_starts = max(1, self.rand_starts_)
        ei_x__best = (0.0, None)
        for iteration in range(self.rand_starts_):
            #generate a random initial guess
            x0 = array([uniform(lower, upper) for lower, upper in self.ranges_])
            #optimize the expected improvement over the valid ranges along each axis
            result = minimize(self.NegativeExpectedImprovement, x0, 
                              method='L-BFGS-B', tol=self.tol_, bounds=self.ranges_, options={'disp': True})
            ei_x = (self.ExpectedImprovement(result.x), result.x)
            if ei_x[0] != ei_x__best[0]:
                ei_x__best = max(ei_x, ei_x__best)
            else:
                ei_x__best = ei_x
        return ei_x__best
    #optimize the fitness and find a good suggested point to try next
    def Suggest(self, num_suggestions = 1, method = 'kg'):      
        #train model
        self.X_virtual_ = []
        self.y_virtual_ = []
        self.gaussian_process_regressor_.fit(self.X_ + self.X_virtual_, self.y_ + self.y_virtual_)
        #use either knowledge gradient or standard approach
        f, x = self.Optimize()
        return x.tolist()
        
        #for multipoint we must add virtual samples and repeat
        #for i in range(num_suggestions - 1):
        #    
        #    x = Optimize(rand_starts=20)
        

In [151]:
#this is the protocol for storing the bayesian optimizer object
#from gaussian_process_regressor_ import GaussianProcessRegressor_
import dill
#from scipy.optimize import anneal
from scipy.optimize import minimize
from numpy import array
from random import random
from numpy import exp, pi, sqrt
from scipy.special import erf
import pathos.multiprocessing as mp
#import scipy

class BayesianOptimizer:
    #if a previous bayesian optimzer was made it can be loaded
    #else we do not do anything
    def __init__(self, filename = None):
        return None     
    #establish the regressor        
    def InitalizeRegressor(self, nu = 1.5):
        self.gaussian_process_regressor_ = GaussianProcessRegressor_(nu = nu)
        return None   
    #set the various simulated annealing parameters    
    def InitializeOptimizer(self, rand_starts=10000, tol=1e-20):
        self.rand_starts_ = rand_starts
        self.tol_ = tol
        None
    #sets the maximum fitness value based on either standard or knowledge gradient
    def SetMaxFitness(self):
        self.y_max_real_and_virtual_ = max(self.y_ + self.y_virtual_)
        return None
    #establish a library of initial samples to seed the model with
    def InitialSamples(self, features, fitness):
        self.X_ = features
        self.y_ = fitness
        self.X_virtual_ = []
        self.y_virtual_ = []
        self.SetMaxFitness()
        return None
    #adds samples - typically this is a result of doing new experiments
    def AddSamples(self, features, fitness):
        self.X_ = self.X_ + features
        self.y_ = self.y_ + fitness
        self.SetMaxFitness()
        return None
    #this controls the range of valid possible values for each axis
    def SetRanges(self, ranges):
        self.ranges_ = ranges
        return None
    #adds some virtual samples - this is relevant to multipoint optimization
    def AddVirtualSamples(self, features, fitness):
        self.X_virtual_ = self.X_virtual_ + features
        self.y_virtual_ = self.y_virtual_ + fitness
        self.SetMaxFitness()
        return None
    #the fitness landscape to optimize and find the best fitness
    def ExpectedImprovement(self, x):
        from numpy import array
        from numpy import exp, pi, sqrt
        from scipy.special import erf
        f_mean, f_stdev  = self.gaussian_process_regressor_.predict(array([x]), return_std = True)
        f_mean = f_mean[0]
        f_stdev = f_stdev[0]
        f_max = self.y_max_real_and_virtual_
        f1 = exp(-((f_max - f_mean)**2.0)/(2.0*f_stdev**2.0))/(2.0*pi)
        f2 = (f_mean - f_max)/(f_stdev*2.0*sqrt(2.0*pi))
        f3 = (f_max - f_mean)*erf((f_max - f_mean)/(sqrt(2.0)*f_stdev))/(2.0*sqrt(2.0*pi)*f_stdev)
        f = f1 + f2 + f3
        #print f
        return f    
    #just the negative of the expected improvement
    def NegativeExpectedImprovement(self, x):
        return -1.0*self.ExpectedImprovement(x)
    
    
    def Minimize_(self, x0):
        result = minimize(self.NegativeExpectedImprovement, x0, 
                              method='L-BFGS-B', bounds=self.ranges_, options={'disp': True})
        return result
    #wrapper for the scipy l-bfgs-b optimizer
    #perform some number of random initializations and take the best result        
    def Optimize(self):
        from scipy.optimize import minimize
        from numpy import array
        p = mp.ProcessingPool(2)
        rand_starts = max(1, self.rand_starts_)
        ei_x__best = (0.0, None)
        x0s = [array([uniform(lower, upper) for lower, upper in self.ranges_]) for iteration in range(self.rand_starts_)]
        minimize_ = lambda q: minimize(self.NegativeExpectedImprovement, q, 
                              method='L-BFGS-B', bounds=self.ranges_, options={'disp': True})
        results = p.map(minimize_, x0s)
        for result in results:
            ei_x = (self.ExpectedImprovement(result.x), result.x)
            try:
                ei_x__best = max(ei_x, ei_x__best)
            except:
                None
        return ei_x__best
    #optimize the fitness and find a good suggested point to try next
    def Suggest(self, num_suggestions = 1, method = 'kg'):      
        #train model
        self.X_virtual_ = []
        self.y_virtual_ = []
        self.gaussian_process_regressor_.fit(self.X_ + self.X_virtual_, self.y_ + self.y_virtual_)
        #use either knowledge gradient or standard approach
        f, x = self.Optimize()
        return x.tolist()
        
        #for multipoint we must add virtual samples and repeat
        #for i in range(num_suggestions - 1):
        #    
        #    x = Optimize(rand_starts=20)
        

In [33]:
from random import uniform

In [34]:
def rosen(x):
    """The Rosenbrock function"""
    return -1.0*sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [35]:
ranges = [(-1.2,1.2), (-1.2,1.2), (-1.2,1.2), (-1.2,1.2)]

In [36]:
features = []
fitness = []
for i in range(20):
    x_rand = array([uniform(lower, upper) for lower, upper in ranges])
    features.append(x_rand.tolist())
    fitness.append(rosen(x_rand))

In [155]:
bayesian_optimizer = BayesianOptimizer()

In [156]:
bayesian_optimizer.InitalizeRegressor()
bayesian_optimizer.InitializeOptimizer(rand_starts = 100, tol=1e-5)

In [157]:
bayesian_optimizer.InitialSamples(features, fitness)

In [158]:
def rosen(x):
    """The Rosenbrock function"""
    return -1.0*sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [159]:
ranges = [(-3,3), (-3,3), (-3,3), (-3,3)]
ranges = [(0.5,1.5), (0.5,1.5), (0.5,1.5), (0.5,1.5)]

In [94]:
features = []
fitness = []
for i in range(20):
    x_rand = array([uniform(lower, upper) for lower, upper in ranges])
    features.append(x_rand.tolist())
    fitness.append(rosen(x_rand))

In [95]:
fitness

[-2.9158679973101065,
 -405.09354154545906,
 -176.87747193066508,
 -132.08779865575417,
 -41.663847691071318,
 -102.21477949174168,
 -53.376407928191497,
 -246.10047446236496,
 -248.49110038697455,
 -33.372852745319733,
 -373.26449696252598,
 -108.52379957404094,
 -22.583267346681772,
 -31.203588359631564,
 -262.85564712964003,
 -46.963031405948669,
 -59.140998948280185,
 -119.22311288406789,
 -27.637184802696545,
 -75.092445147766014]

In [64]:
bayesian_optimizer.InitialSamples(features, fitness)

In [65]:
bayesian_optimizer.SetRanges(ranges)

In [66]:
bayesian_optimizer.Suggest()

0


C:\Users\rbj428\AppData\Local\Continuum\Anaconda2_3\lib\site-packages\ipykernel\__main__.py:83: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


[1.0328136676628734, 1.0661435986679049, 1.1351630443366159, 1.210582393381338]

In [67]:
bayesian_optimizer.Suggest()

0


C:\Users\rbj428\AppData\Local\Continuum\Anaconda2_3\lib\site-packages\ipykernel\__main__.py:83: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


[1.0329008260431334, 1.0659563471185693, 1.1357148022031975, 1.207764674191187]

In [353]:
bayesian_optimizer.Suggest()

0
100
200
300
400
500
600
700
800
900


[0.21263814533992242,
 1.4324832371853715,
 -0.3825618979108294,
 -2.943100426444561]

In [ ]:
def rosen(x):
    """The Rosenbrock function"""
    return -1.0*sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [ ]:
ranges = [(0.5,1.5), (0.5,1.5), (0.5,1.5), (0.5,1.5)]
features = []
fitness = []
for i in range(20):
    x_rand = array([uniform(lower, upper) for lower, upper in ranges])
    features.append(x_rand.tolist())
    fitness.append(rosen(x_rand))

In [192]:
bayesian_optimizer = BayesianOptimizer()
bayesian_optimizer.InitalizeRegressor()
bayesian_optimizer.InitializeOptimizer(rand_starts = 50, tol=1e-5)

In [195]:
bayesian_optimizer.InitialSamples(features, fitness)
bayesian_optimizer.SetRanges(ranges)
xs = []
ys = []
start_time = timeit.default_timer()
for i in range(30):
    print 'iteration %i' % i
    x_new = bayesian_optimizer.Suggest()
    y_new = rosen(array(x_new))
    xs.append(x_new)
    ys.append(y_new)
    bayesian_optimizer.AddSamples([x_new], [y_new])
elapsed = timeit.default_timer() - start_time
print elapsed

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
82.099714516


In [194]:
ys

[-0.97412586483487407,
 -1.1617567133221414,
 -2.0402741460216887,
 -0.94737208422844277,
 -1.0160048961196888,
 -2.1444777541949547,
 -11.870687109830396,
 -2.28594618452024,
 -1.1780902341620663,
 -2.0140915244806137,
 -2.1154318557925329,
 -1.1475572460862229,
 -1.4254802655134409,
 -0.91559733448718439,
 -12.206294599527551,
 -0.96843771393635203,
 -1.158992627010927,
 -0.91317583102853395,
 -2.1689332504226,
 -8.3226027279139316,
 -0.99123828192265773,
 -0.92690233752421047,
 -1.6192679260779135,
 -0.93340560675377948,
 -3.0528452441386147,
 -1.0709224918647269,
 -0.98269381828860292,
 -2.979254954520564,
 -1.1635390016245482,
 -10.398620147457272]

In [196]:
ys

[-2.9219635149796517,
 -6.349551394265136,
 -4.6354396447215294,
 -2.0524680001741746,
 -19.35207032885782,
 -1.6910402168825045,
 -1.3993413223115341,
 -1.2153547610681636,
 -1.1093424084284649,
 -1.0470096932019426,
 -0.97976011569888377,
 -0.9089691700206769,
 -0.84732474575071026,
 -0.81263529734007078,
 -0.84635335773580789,
 -0.77119014099202365,
 -0.7252757917046404,
 -0.66827598091659102,
 -0.57964750271366927,
 -0.47108446862746639,
 -0.38322353358714167,
 -0.34352868596830499,
 -0.33722357500415906,
 -0.33922169550719911,
 -0.33955607656291315,
 -0.33956583946531538,
 -0.33753498015505023,
 -0.33940329432046268,
 -0.33639939662220314,
 -0.339442749182429]

In [178]:
max(fitness)

-29.485683241212509

In [188]:
ys

[-1.6083908786652357,
 -8.1138135095277697,
 -1.0080787765535961,
 -12.676456084310898,
 -1.2139096063411134,
 -1.1151256110182823,
 -1.1854025257066576,
 -0.90277628027896661,
 -0.80423418596446261,
 -0.72693478385668764,
 -0.69073338561955622,
 -0.70583857523994276,
 -0.756643378683581,
 -0.66123714173282333,
 -0.60134417394235573,
 -0.50854257788608037,
 -0.46624722637449123,
 -0.4836355147349376,
 -0.45473310940065087,
 -0.57256976484278699,
 -0.50138223886934552,
 -0.51884028223605161,
 -0.35476792609533825,
 -0.45857083292988687,
 -0.71138880208979804,
 -2.6680289711555365,
 -0.8090432187818688,
 -0.32631234754516386,
 -0.77825194503468453,
 -0.32422075071326334]

In [146]:
import timeit

In [162]:
bayesian_optimizer.InitialSamples(features, fitness)
bayesian_optimizer.SetRanges(ranges)
xs = []
ys = []
start_time = timeit.default_timer()
for i in range(2):
    print 'iteration %i' % i
    x_new = bayesian_optimizer.Suggest()
    y_new = rosen(array(x_new))
    xs.append(x_new)
    ys.append(y_new)
    bayesian_optimizer.AddSamples([x_new], [y_new])
elapsed = timeit.default_timer() - start_time
print elapsed

iteration 0
0
100
200
300
400
500
600
700
800
900
iteration 1
0
100
200
300
400
500
600
700
800
900
58.5018798046


In [150]:
bayesian_optimizer.InitialSamples(features, fitness)
bayesian_optimizer.SetRanges(ranges)
xs = []
ys = []
start_time = timeit.default_timer()
for i in range(2):
    print 'iteration %i' % i
    x_new = bayesian_optimizer.Suggest()
    y_new = rosen(array(x_new))
    xs.append(x_new)
    ys.append(y_new)
    bayesian_optimizer.AddSamples([x_new], [y_new])
elapsed = timeit.default_timer() - start_time
print elapsed

iteration 0
iteration 1
98.2092614918


In [153]:
bayesian_optimizer.InitialSamples(features, fitness)
bayesian_optimizer.SetRanges(ranges)
xs = []
ys = []
start_time = timeit.default_timer()
for i in range(2):
    print 'iteration %i' % i
    x_new = bayesian_optimizer.Suggest()
    y_new = rosen(array(x_new))
    xs.append(x_new)
    ys.append(y_new)
    bayesian_optimizer.AddSamples([x_new], [y_new])
elapsed = timeit.default_timer() - start_time
print elapsed

iteration 0
iteration 1
89.5332763313


In [107]:
ys

[-2.4139698697553706,
 -2.7183738179399528,
 -4.1915549096068343,
 -1.5070386349609499,
 -3.686148217241231,
 -128.9981894121795,
 -0.98910660410109996,
 -0.69782958416594765,
 -0.6129857957606325,
 -0.6450511636618399,
 -0.5549724762644217,
 -0.49680064107290328,
 -0.49422405613349268,
 -0.50193302175962373,
 -0.4877607144700703,
 -0.45749341312611586,
 -0.35822169765243028,
 -0.26677099814555638,
 -0.2167260400984678,
 -0.19442497651580026,
 -0.18505052869507754,
 -0.18264075938024923,
 -1.1024884450958856,
 -0.17455004265482177,
 -0.17522627453830708,
 -0.17413990224668996,
 -0.17327388502196162,
 -0.17535776267643005,
 -0.17481111074476513,
 -0.17432689000537466]

In [108]:
fitness

[-2.9158679973101065,
 -405.09354154545906,
 -176.87747193066508,
 -132.08779865575417,
 -41.663847691071318,
 -102.21477949174168,
 -53.376407928191497,
 -246.10047446236496,
 -248.49110038697455,
 -33.372852745319733,
 -373.26449696252598,
 -108.52379957404094,
 -22.583267346681772,
 -31.203588359631564,
 -262.85564712964003,
 -46.963031405948669,
 -59.140998948280185,
 -119.22311288406789,
 -27.637184802696545,
 -75.092445147766014]

In [23]:
bayesian_optimizer.InitialSamples(features, fitness)
ranges = [(-3,3), (-3,3), (-3,3), (-3,3)]
bayesian_optimizer.SetRanges(ranges)
xs = []
ys = []
for i in range(60):
    print 'iteration %i' % i
    x_new = bayesian_optimizer.Suggest()
    y_new = rosen(array(x_new))
    xs.append(x_new)
    ys.append(y_new)
    bayesian_optimizer.AddSamples([x_new], [y_new])
    

iteration 0
0
iteration 1
0
iteration 2
0
iteration 3
0
iteration 4
0
iteration 5
0
iteration 6
0
iteration 7
0
iteration 8
0
iteration 9
0
iteration 10
0


C:\Users\rbj428\AppData\Local\Continuum\Anaconda2_3\lib\site-packages\ipykernel\__main__.py:83: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


iteration 11
0
iteration 12
0
iteration 13
0
iteration 14
0
iteration 15
0
iteration 16
0
iteration 17
0
iteration 18
0
iteration 19
0
iteration 20
0
iteration 21
0
iteration 22
0
iteration 23
0
iteration 24
0
iteration 25
0
iteration 26
0
iteration 27
0
iteration 28
0
iteration 29
0
iteration 30
0
iteration 31
0
iteration 32
0
iteration 33
0
iteration 34
0
iteration 35
0
iteration 36
0
iteration 37
0
iteration 38
0
iteration 39
0
iteration 40
0
iteration 41
0
iteration 42
0
iteration 43
0
iteration 44
0
iteration 45
0
iteration 46
0
iteration 47
0
iteration 48
0
iteration 49
0
iteration 50
0
iteration 51
0
iteration 52
0
iteration 53
0
iteration 54
0
iteration 55
0
iteration 56
0
iteration 57
0
iteration 58
0
iteration 59
0


In [24]:
ys

[-304.39007831532189,
 -469.10196213697543,
 -209.8365429316288,
 -489.59400940037898,
 -82.935374986250594,
 -58.127665884707618,
 -53.88587076278462,
 -48.447066968979748,
 -44.164713459141467,
 -251.39595899441636,
 -40.522825094375392,
 -37.84423185601058,
 -36.64828561594863,
 -36.543412211227199,
 -36.735974047688707,
 -35.973050418609475,
 -34.261709042540858,
 -32.478573778850418,
 -30.776573367034267,
 -29.239381751157929,
 -28.292760571163072,
 -28.122683968118707,
 -28.245467919039029,
 -27.977827695738913,
 -27.933140471175676,
 -27.955676144574387,
 -28.039975930959216,
 -27.856535820102,
 -28.501916659398191,
 -27.145761670697087,
 -26.547207834407235,
 -26.495099441235705,
 -26.882614378588478,
 -25.166211090240456,
 -22.784114773346978,
 -20.968069177632561,
 -19.585153817299627,
 -18.935058393643011,
 -18.894855074051534,
 -65.980975263068316,
 -185.87559295519947,
 -18.596024870272707,
 -18.858348077644411,
 -17.833716804242716,
 -16.807779616888769,
 -16.357352402002

In [523]:
xs

[[0.8326930444563336,
  0.28804476048531086,
  1.0574907567062508,
  -0.018285997713875093],
 [0.8716733693058609,
  0.30110352815475,
  1.0595335148465623,
  0.0038769146390718445],
 [-0.26302539857205337,
  0.25923235653249305,
  1.0576408734076457,
  0.02652052919665926],
 [-2.0956113746357516,
  0.1650526687093348,
  1.0539634994828606,
  0.09235500462660617],
 [-0.1068679970390355,
  0.28818271328419404,
  1.0585974415942176,
  0.020415387341477687],
 [1.684746919977236,
  1.2716109379798395,
  0.6785613781448621,
  -1.4668685126554173],
 [-0.11905987500258527,
  0.2443333086466741,
  1.0604785581131475,
  0.03273533602776593],
 [-0.10088617019774099,
  0.44290588069557746,
  1.0550725127128584,
  0.008495340657700808],
 [0.47855253035720724,
  -1.2679359076776868,
  1.0298956776586528,
  0.021770515555093503],
 [-0.09748462286221145,
  1.8470392713031536,
  1.058891348901641,
  -0.014996808665569017],
 [0.3711563328197094,
  0.07043997380332667,
  1.0264595983394302,
  0.01048644

In [408]:
rosen(array([0.98,1.01,1.01,0.98]))

-0.4176180000000011

In [436]:
import pathos.multiprocessing as mp

In [32]:
zip(*xs)[1]

(-0.8856658834479298,
 0.7794761429062657,
 -1.0753451740209397,
 -0.6699516428775749,
 0.5220336657592312,
 0.16722006293693353,
 0.14579379375695053,
 0.11719411989555382,
 0.09064355023544096,
 1.2585596589101635,
 0.06692481579103074,
 0.043324635053183895,
 0.021413812973911213,
 0.0033767123114945333,
 0.005095967608958329,
 -0.0035189676682394985,
 -0.030262680635309114,
 -0.056320763546144736,
 -0.08049092356348628,
 -0.10488225772430645,
 -0.1273837563889018,
 -0.14916247593479987,
 -0.1553282003138263,
 -0.14005315283867195,
 -0.12846356533213216,
 -0.12640904109707274,
 -0.13206819661291513,
 -0.1210462182610965,
 -0.12395464963211089,
 -0.1266391988351735,
 -0.11775939617268753,
 -0.11118249716670756,
 -0.12433217058098159,
 -0.10155735405620687,
 -0.07898850103230913,
 -0.0613029675343736,
 -0.04127708759351171,
 -0.014496359744930621,
 0.007625341124839893,
 0.5234549835681406,
 0.8918135052984955,
 -0.002358613753894017,
 -0.00226874102109235,
 -0.01293299916360551,
 -0.

In [253]:
array([array([3,4,5])]).tolist()

[[3, 4, 5]]

In [63]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [ ]:
bayesian_optimizer.InitialSamples(features, fitness)

In [3]:
ranges = [(-4,6),(-100,0),(0,2),(5,9)]

In [4]:
from numpy import array
from random import uniform

In [5]:
x0 = array([uniform(lower, upper) for lower, upper in ranges])

In [6]:
x0

array([  2.6489971 , -83.05540728,   0.76393971,   8.68494422])

In [220]:
import numpy as np
from scipy.optimize import minimize

In [221]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [231]:
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
res = minimize(rosen, x0, method='L-BFGS-B', bounds=[(-1,2), (0.6,0.9), (0, 9),(-1,2), (-1,2)], tol=1e-30, options={'disp': False})
print 'hi'

hi


In [232]:
res.x

array([ 0.94882541,  0.9       ,  0.81720332,  0.67111015,  0.45038883])

In [233]:
res


      fun: 0.16047972744778483
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -5.55111512e-09,  -2.84712449e+00,   3.44169138e-07,
         4.41313652e-07,   4.38538095e-07])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 582
      nit: 26
   status: 0
  success: True
        x: array([ 0.94882541,  0.9       ,  0.81720332,  0.67111015,  0.45038883])

In [ ]:
def Optimize(self, rand_starts = 10000):
        rand_starts = max(1, rand_starts)
        ei_x__best = (0.0, None)
        for iteration in range(rand_starts):
            #generate a random initial guess
            x0 = array([uniform(lower, upper) for lower, upper in self.ranges_])
            #optimize the expected improvement over the valid ranges along each axis
            result = anneal(self.NegativeExpectedImprovement, x0, schedule='boltzmann',
                          full_output=False, maxiter=500, lower=self.lower_,
                          upper=self.upper_, dwell=250, disp=True)
            ei_x = (-1.0*result[1], result[0])
            ei_x__best = max(ei_x, ei_x__best)
        return ei_x__best